# Multi-agent Configuration Exploration

This notebook explores composition of developed agents to get the desired output for GAIA tasks. 

In [ ]:
%pip install smolagents
%pip install devtools

from dotenv import load_dotenv
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))
load_dotenv()

## Final Answer Formatting

In [ ]:
from smolagents import Model, FinalAnswerTool
from devtools import pprint


class RefineGAIAAnswer(FinalAnswerTool):
    name = "refine_answer"  # workaround since smolagents now does not call final tool
    description = "Refines and replaces final answer. It must always be called before giving the final answer."
    inputs = {
        "task": {"type": "string", "description": "The original problem to solve."},
        "answer": {"type": "string", "description": "The final answer to the problem."},
    }
    output_type = "string"

    _system_prompt = """\
You are a general AI assistant.

I will provide you a question and an answer to it.
Your task is:
- refine the answer and ensure it is what was asked in the question.
- rephrase answer according to rules below.

Answer should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. 

If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. 
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. 
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.
If you are asked for a comma separated list use space after comma and before next element of the list unless other directly specified in a question.
If answer does not enforce specific format, capitalize the first letter of the answer. Check question context to define if letters case matters.
If answer is number use digits only not words unless other directly specified in a question.
If answer is not full sentence, do not add period at the end.
"""

    def __init__(self, model: Model, **kwargs):
        self.model = model
        super().__init__(**kwargs)

    def forward(self, task: str, answer: str) -> str:
        if not answer:
            return answer

        response = self.model(
            [
                {"role": "system", "content": self._system_prompt},
                {"role": "user", "content": self._prompt(task, answer)},
            ]
        )

        return response.content

    def _prompt(self, task: str, answer: str) -> str:
        return f"""\
QUESTION:
{task}

ANSWER:
{answer}
"""


from smolagents import OpenAIServerModel

final_answer_tool = RefineGAIAAnswer(model=OpenAIServerModel(model_id="gpt-4.1"))

# final_answer = final_answer_tool(
#     task="""\
# In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?
# """,
#     answer="""\
# The highest number of bird species to be on camera simultaneously in the video is three: Emperor
# Penguin chicks, Giant Petrel, and Adelie Penguin.
# """,
# )
# print(final_answer)

# final_answer = final_answer_tool(
#     task="""\
# Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec. What does Teal'c say in response to the question \"Isn't that hot?\"
# """,
#     answer="""\
# In response to the question "Isn't that hot?", Teal'c says, "Extremely."
# """,
# )
# print(final_answer)

# final_answer = final_answer_tool(
#     task="""\
# The attached Excel file contains the sales of menu items for a local fast-food chain. What were the total sales that the chain made from food (not including drinks)? Express your answer in USD with two decimal places.
# """,
#     answer="""\
# $89706.00
# """,
# )
# print(final_answer)

# final_answer = final_answer_tool(
#     task="""\
# What peice is on d8 square in the chess position from the image? Give the piece code only.
# """,
#     answer="r",  # important to keep piece code lower to denote that rook is black
# )
# print(final_answer)

# final_answer = final_answer_tool(
#     task="""\
# List the pages to review in ascending order.
# """,
#     answer="123,231,199",
# )
# print(final_answer)

final_answer = final_answer_tool(
    task="""\
.rewsna eht sa "tfel" drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI
""",
    answer="""\
If you understand this sentence, write the opposite of the word "left" as the answer.
""",
)
print(final_answer)

## Agents Assembling

In [ ]:
from smolagents import CodeAgent, ToolCallingAgent, OpenAIServerModel, tool
from tools import (
    GetAttachmentTool,
    GoogleSearchTool,
    GoogleSiteSearchTool,
    ContentRetrieverTool,
    SpeechRecognitionTool,
    YoutubeVideoTool,
    ImageToChessBoardFENTool,
    chess_engine_locator,
)

model = OpenAIServerModel(model_id="gpt-4.1")
verbosity = 2

get_attachment_tool = GetAttachmentTool()
env_tools = [
    get_attachment_tool,
]

researcher_agent = ToolCallingAgent(
    name="web_researcher",
    description="Browses the web to find information.",
    tools=[
        GoogleSearchTool(),
        GoogleSiteSearchTool(),
        ContentRetrieverTool(),
    ],  # no env_tools!
    model=model,
    planning_interval=3,
    max_steps=9,
    verbosity_level=verbosity,
)


@tool
def web_search(task: str) -> str:
    """
    Solves problems requiring information search in the internet. High priority if applicable.
    Args:
        task: The original task without any changes.
    Returns:
        The answer to the question.
    """
    return researcher_agent.run(task)


data_analyst_agent = CodeAgent(
    name="data_analyst",
    description="Data analyst with advanced skills in statistic, handling tabular data and related Python packages.",
    tools=env_tools,
    additional_authorized_imports=[
        "numpy",
        "pandas",
        "tabulate",
        "matplotlib",
        "seaborn",
    ],
    model=model,
    verbosity_level=verbosity,
)


@tool
def data_analysis(task: str) -> str:
    """
    Analyses excel, csv and other tabular data. High priority if applicable.
    Args:
        task: The original task without any changes.
    Returns:
        The answer to the question.
    """
    return data_analyst_agent.run(task)


chess_player_agent = CodeAgent(
    name="chess_player",
    description="Chess grandmaster empowered by chess engine. Always thinks at least 100 steps ahead.",
    tools=env_tools
    + [
        ImageToChessBoardFENTool(client=model.client),
        chess_engine_locator,
    ],
    additional_authorized_imports=[
        "chess",
        "chess.engine",
    ],
    model=model,
    verbosity_level=verbosity,
)

speech_recognition_tool = SpeechRecognitionTool()


@tool
def chess_problem_solving(task: str) -> str:
    """
    Solves chess problems. High priority if applicable.
    Args:
        task: The original task without any changes.
    Returns:
        The answer to the question.
    """
    return chess_player_agent.run(task)


In [ ]:
manager_agent = ToolCallingAgent(
    name="general_agent",
    description="Answers questions with help of tools and best of knowledge.",
    model=model,
    tools=env_tools
    + [
        speech_recognition_tool,
        YoutubeVideoTool(
            client=model.client,
            speech_recognition_tool=speech_recognition_tool,
        ),
        web_search,
        data_analysis,
        chess_problem_solving,
        RefineGAIAAnswer(model=model),
    ],
    max_steps=12,
    verbosity_level=verbosity,
)

Notes:
- FinalTool is not called for ToolCallingAgent, it should be added separately.
- Agent sometimes ignore refined answer and return original answer.
- Custom logic is needed to adapt for GAIA challenge.

## Custom Agent Implementation

In [ ]:
from smolagents import Text, LogLevel


class GeneralAgent(ToolCallingAgent):
    def run(self, task: str, stream: bool = False, **kwargs):
        if stream:
            raise NotImplementedError("Streaming is not supported by GeneralAgent.")

        answer = super().run(task=task, stream=stream, **kwargs)
        refined_answer = self._refine_answer(task, answer)
        if refined_answer != answer:
            self._publish_refined_answer(refined_answer)

        return refined_answer

    def _refine_answer(self, task: str, answer: str):
        response = self.model(
            [
                {"role": "system", "content": self._refinement_system_prompt},
                {"role": "user", "content": self._refinement_prompt(task, answer)},
            ]
        )

        return response.content

    def _publish_refined_answer(self, refined_answer):
        self.logger.log(
            Text(f"Refined GAIA answer: {refined_answer}", style="bold #d4b702"),
            level=LogLevel.INFO,
        )

    _refinement_system_prompt = """\
You are a general AI assistant.

I will provide you a question and an answer to it.
Your task is:
- refine the answer and ensure it is what was asked in the question.
- rephrase answer according to rules below.

Answer should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. 

If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. 
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. 
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.
If you are asked for a comma separated list use space after comma and before next element of the list unless other directly specified in a question.
If answer does not enforce specific format, capitalize the first letter of the answer. Check question context to define if letters case matters.
If answer is number use digits only not words unless other directly specified in a question.
If answer is not full sentence, do not add period at the end.
"""

    def _refinement_prompt(self, task: str, answer: str) -> str:
        return f"""\
QUESTION:
{task}

ANSWER:
{answer}
"""


manager_agent = GeneralAgent(
    name="general_agent",
    description="Answers questions with help of tools and best of knowledge.",
    model=model,
    tools=env_tools
    + [
        speech_recognition_tool,
        YoutubeVideoTool(
            client=model.client,
            speech_recognition_tool=speech_recognition_tool,
        ),
        web_search,
        data_analysis,
        chess_problem_solving,
    ],
    max_steps=12,
    verbosity_level=verbosity,
)

### Simple Task

In [ ]:
get_attachment_tool.attachment_for("2d83110e-a098-4ebb-9987-066c06fa42d0")
manager_agent.run("""\
.rewsna eht sa "tfel" drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI
""");

### Audition

In [ ]:
get_attachment_tool.attachment_for("1f975693-876d-457b-a649-393859e79bf3")
manager_agent.run("""\
Hi, I was out sick from my classes on Friday, so I'm trying to figure out what I need to study for my Calculus mid-term next week. My friend from class sent me an audio recording of Professor Willowbrook giving out the recommended reading for the test, but my headphones are broken :(
Could you please listen to the recording for me and tell me the page numbers I'm supposed to go over? I've attached a file called Homework.mp3 that has the recording. Please provide just the page numbers as a comma-delimited list. And please provide the list in ascending order.
""");

### YouTube Video

In [ ]:
get_attachment_tool.attachment_for(
    "a1e91b78-d3d8-4675-bb8d-62741b4b68a6"
)  # no attachment
manager_agent.run(
    "In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?"
);

### Data Analysis

In [ ]:
get_attachment_tool.attachment_for("7bd855d8-463d-4ed5-93ca-5fe35145f733")
manager_agent.run(
    "The attached Excel file contains the sales of menu items for a local fast-food chain. What were the total sales that the chain made from food (not including drinks)? Express your answer in USD with two decimal places."
);

### Web Search

In [ ]:
get_attachment_tool.attachment_for(
    "8e867cd7-cff9-4e6c-867a-ff5ddc2550be"
)  # no attachment
manager_agent.run(
    "How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia."
);

### Chess Problem

In [ ]:
get_attachment_tool.attachment_for("cca530fc-4052-43b2-b130-b30968d8aa44")
manager_agent.run(
    "Review the chess position provided in the image. It is black's turn. Provide the correct next move for black which guarantees a win. Please provide your response in algebraic notation."
);

## Agents Graph

In [ ]:
%pip install langgraph langchain_openai backoff

In [ ]:
from smolagents import Tool
from openai import OpenAI


class ClassifierTool(Tool):
    name = "open_classifier"
    description = """Classifies given items into given categories from perspective of specific knowledge area."""
    inputs = {
        "knowledge_area": {
            "type": "string",
            "description": "The knowledge area that should be used for classification.",
        },
        "environment": {  # context make models too verbose
            "type": "string",
            "description": "Couple words that describe environment or location in which items should be classified in case of plural meaning or if only part of item relevant for classification.",
        },
        "categories": {
            "type": "string",
            "description": "Comma separated list of categories to distribute objects.",
        },
        "items": {
            "type": "string",
            "description": "Comma separated list of items to be classified. Please include adjectives if available.",
        },
    }
    output_type = "string"

    def __init__(
        self,
        client: OpenAI | None = None,
        model_id: str = "gpt-4.1-mini",
        **kwargs,
    ):
        self.client = client or OpenAI()
        self.model_id = model_id

        super().__init__(**kwargs)

    def forward(
        self, knowledge_area: str, environment: str, categories: str, items: str
    ) -> str:
        response = self.client.responses.create(
            model=self.model_id,
            input=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "input_text",
                            "text": self._prompt(
                                knowledge_area=knowledge_area,
                                context=environment,
                                categories=categories,
                                items=items,
                            ),
                        },
                    ],
                }
            ],
        )
        answer = response.output_text
        return answer

    def _prompt(
        self, knowledge_area: str, context: str, categories: str, items: str
    ) -> str:
        return f"""\
You are {knowledge_area} classifier located in {context} context.
I will provide you a list of items and a list of categories and context in which items should be considered.

Your task is to classify the items into the categories.
Use context to determine the meaning of the items and decide if you need to classify entire item or only part of it.

Do not miss any item and do not add any item to the list of categories.
Use highest probability category for each item.
You can add category "Other" if you are not sure about the classification.

Use only considerations from from the {knowledge_area} perspective.
Explain your reasoning from {knowledge_area} perspective in {context} context and then provide final answer.
Important: Do not allow {context} influence your judgment for classification.

ITEMS: {items}
CATEGORIES: {categories}

Now provide your reasoning and finalize it with the classification in the following format:
Category 1: items list
Category 2: items list
Other (if needed): items list
"""


classifier_tool = ClassifierTool(client=OpenAI(), model_id="gpt-4.1")
a = classifier_tool(
    knowledge_area="botanic",
    environment="grocery list",
    categories="vegetables",
    items="milk, eggs, flour, whole bean coffee, Oreos, sweet potatoes, fresh basil, plums, green beans, rice, corn, bell pepper, whole allspice, acorns, broccoli, celery, zucchini, lettuce, peanuts",
)

print(a)

In [ ]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from rich.console import Console
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    OpenAIServerModel,
    AgentLogger,
    LogLevel,
    Panel,
    Text,
)
from tools import (
    GetAttachmentTool,
    GoogleSearchTool,
    GoogleSiteSearchTool,
    ContentRetrieverTool,
    YoutubeVideoTool,
    SpeechRecognitionTool,
    ImageToChessBoardFENTool,
    chess_engine_locator,
)
import openai
import backoff
from IPython.display import Image, display

verbosity = LogLevel.DEBUG
get_attachment_tool = GetAttachmentTool()
speech_recognition_tool = SpeechRecognitionTool()
env_tools = [
    get_attachment_tool,
]
model = OpenAIServerModel(model_id="gpt-4.1")
console = Console(record=True)
logger = AgentLogger(level=verbosity, console=console)
steps_buffer = []


def capture_step_log(agent) -> None:
    steps_buffer.append(console.export_text(clear=True))


agents = {
    agent.name: agent
    for agent in [
        ToolCallingAgent(
            name="general_assistant",
            description="Answers questions for best of knowledge and common reasoning grounded on already known information. Can understand multimedia including audio and video files and YouTube.",
            model=model,
            tools=env_tools
            + [
                speech_recognition_tool,
                YoutubeVideoTool(
                    client=model.client,
                    speech_recognition_tool=speech_recognition_tool,
                    frames_interval=3,
                    chunk_duration=60,
                    debug=True,
                ),
                ClassifierTool(
                    client=model.client,
                    model_id="gpt-4.1-mini",
                ),
            ],
            logger=logger,
            step_callbacks=[capture_step_log],
        ),
        ToolCallingAgent(
            name="web_researcher",
            description="Answers questions that require grounding in unknown information through web search, sites, and other online resources (e.g. wikipedia.org, arxiv.org, etc.).",
            tools=env_tools
            + [
                GoogleSearchTool(),
                GoogleSiteSearchTool(),
                ContentRetrieverTool(),
            ],
            model=model,
            planning_interval=3,
            max_steps=9,
            logger=logger,
            step_callbacks=[capture_step_log],
        ),
        CodeAgent(
            name="data_analyst",
            description="Data analyst with advanced skills in statistic, handling tabular data and related Python packages.",
            tools=env_tools,
            additional_authorized_imports=[
                "numpy",
                "pandas",
                "tabulate",
                "matplotlib",
                "seaborn",
            ],
            model=model,
            logger=logger,
            step_callbacks=[capture_step_log],
        ),
        CodeAgent(
            name="chess_player",
            description="Chess grandmaster empowered by chess engine. Always thinks at least 100 steps ahead.",
            tools=env_tools
            + [
                ImageToChessBoardFENTool(client=model.client),
                chess_engine_locator,
            ],
            additional_authorized_imports=[
                "chess",
                "chess.engine",
            ],
            model=model,
            logger=logger,
            step_callbacks=[capture_step_log],
        ),
    ]
}


class GAIATask(TypedDict):
    task_id: Optional[str | None] = None
    question: str
    steps: list[str] = []
    agent: Optional[str | None] = None
    raw_answer: Optional[str | None] = None
    final_answer: Optional[str | None] = None


llm = ChatOpenAI(model="gpt-4.1")
logger = AgentLogger(level=verbosity)


@backoff.on_exception(backoff.expo, openai.RateLimitError, max_time=60, max_tries=6)
def llm_invoke_with_retry(messages):
    response = llm.invoke(messages)
    return response


def read_question(state: GAIATask):
    logger.log_task(
        content=state["question"].strip(),
        subtitle=f"LangGraph with {type(llm).__name__} - {llm.model_name}",
        level=LogLevel.INFO,
        title="Final Assignment Agent for Hugging Face Agents Course",
    )
    get_attachment_tool.attachment_for(state["task_id"])

    return {
        "steps": [],
        "agent": None,
        "raw_answer": None,
        "final_answer": None,
    }


def select_agent(state: GAIATask):
    agents_description = "\n\n".join(
        [
            f"AGENT NAME: {a.name}\nAGENT DESCRIPTION: {a.description}"
            for a in agents.values()
        ]
    )

    prompt = f"""\
You are a general AI assistant.

I will provide you a question and a list of agents with their descriptions.
Your task is to select the most appropriate agent to answer the question.
You can select one of the agents or decide that no agent is needed.

If question has attachment only agent can answer it.

QUESTION:
{state["question"]}

{agents_description}

Now, return the name of the agent you selected or "no agent needed" if you think that no agent is needed.
"""

    response = llm_invoke_with_retry([HumanMessage(content=prompt)])
    agent_name = response.content.strip()

    if agent_name in agents:
        logger.log(
            f"Agent {agent_name} selected for solving the task.",
            level=LogLevel.DEBUG,
        )
        return {
            "agent": agent_name,
            "steps": state.get("steps", [])
            + [
                f"Agent '{agent_name}' selected for task execution.",
            ],
        }
    elif agent_name == "no agent needed":
        logger.log(
            "No appropriate agent found in the list. No agent will be used.",
            level=LogLevel.DEBUG,
        )
        return {
            "agent": None,
            "steps": state.get("steps", [])
            + [
                "A decision is made to solve the task directly without invoking any agent.",
            ],
        }
    else:
        logger.log(
            f"[bold red]Warning to user: Unexpected agent name '{agent_name}' selected. No agent will be used.[/bold red]",
            level=LogLevel.INFO,
        )
        return {
            "agent": None,
            "steps": state.get("steps", [])
            + [
                f"Attempt to select non-existing agent '{agent_name}'. No agent will be used.",
            ],
        }


def delegate_to_agent(state: GAIATask):
    agent_name = state.get("agent", None)
    if not agent_name:
        raise ValueError("Agent not selected.")
    if agent_name not in agents:
        raise ValueError(f"Agent '{agent_name}' is not available.")

    logger.log(
        Panel(Text(f"Calling agent: {agent_name}.")),
        level=LogLevel.INFO,
    )

    agent = agents[agent_name]
    agent_answer = agent.run(task=state["question"])
    steps = [f"Agent '{agent_name}' step:\n{s}" for s in steps_buffer]
    steps_buffer.clear()
    return {
        "raw_answer": agent_answer,
        "steps": state.get("steps", []) + steps,
    }


def one_shot_answering(state: GAIATask):
    response = llm_invoke_with_retry([HumanMessage(content=state.get("question"))])
    return {
        "raw_answer": response.content,
        "steps": state.get("steps", [])
        + [
            f"One-shot answer:\n{response.content}",
        ],
    }


def refine_answer(state: GAIATask):
    question = state.get("question")
    answer = state.get("raw_answer", None)
    if not answer:
        return {"final_answer": "No answer."}

    prompt = f"""\
You are a general AI assistant.

I will provide you a question and correct answer to it. Answer is correct but may be too verbose or not follow the rules below.
Your task is to rephrase answer according to rules below.

Answer should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. 

If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. 
If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. 
If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.

If you are asked for a comma separated list, use space after comma and before next element of the list unless other directly specified in a question.
Check question context to define if letters case matters. Do not change case if not prescribed by other rules or question.
If you are not asked for the list, capitalize the first letter of the answer unless it changes meaning of the answer.
If answer is number, use digits only not words unless other directly specified in a question.
If answer is not full sentence, do not add period at the end.

Preserve all items if the answer is a list.

QUESTION:
{question}

ANSWER:
{answer}
"""
    response = llm_invoke_with_retry([HumanMessage(content=prompt)])
    refined_answer = response.content.strip()
    logger.log(
        Text(f"GAIA final answer: {refined_answer}", style="bold #d4b702"),
        level=LogLevel.INFO,
    )
    return {
        "final_answer": refined_answer,
        "steps": state.get("steps", [])
        + [
            "Refining the answer according to GAIA benchmark rules.",
            f"FINAL ANSWER: {response.content}",
        ],
    }


def route_task(state: GAIATask) -> str:
    if state.get("agent") in agents:
        return "agent selected"
    else:
        return "no agent matched"


# Create the graph
gaia_graph = StateGraph(GAIATask)

# Add nodes
gaia_graph.add_node("read_question", read_question)
gaia_graph.add_node("select_agent", select_agent)
gaia_graph.add_node("delegate_to_agent", delegate_to_agent)
gaia_graph.add_node("one_shot_answering", one_shot_answering)
gaia_graph.add_node("refine_answer", refine_answer)

# Start the edges
gaia_graph.add_edge(START, "read_question")
# Add edges - defining the flow
gaia_graph.add_edge("read_question", "select_agent")

# Add conditional branching from select_agent
gaia_graph.add_conditional_edges(
    "select_agent",
    route_task,
    {"agent selected": "delegate_to_agent", "no agent matched": "one_shot_answering"},
)

# Add the final edges
gaia_graph.add_edge("delegate_to_agent", "refine_answer")
gaia_graph.add_edge("one_shot_answering", "refine_answer")
gaia_graph.add_edge("refine_answer", END)

gaia = gaia_graph.compile()
display(Image(gaia.get_graph().draw_mermaid_png()))

### Simple Task

In [ ]:
task = gaia.invoke(
    {
        "task_id": "2d83110e-a098-4ebb-9987-066c06fa42d0",
        "question": '.rewsna eht sa "tfel" drow eht fo etisoppo eht etirw ,ecnetnes siht dnatsrednu uoy fI',
    }
)

print(task["final_answer"])

In [ ]:
print("\n".join(task["steps"]))

### Audition

In [ ]:
gaia.invoke(
    {
        "task_id": "1f975693-876d-457b-a649-393859e79bf3",
        "question": """\
Hi, I was out sick from my classes on Friday, so I'm trying to figure out what I need to study for my Calculus mid-term next week. My friend from class sent me an audio recording of Professor Willowbrook giving out the recommended reading for the test, but my headphones are broken :(
Could you please listen to the recording for me and tell me the page numbers I'm supposed to go over? I've attached a file called Homework.mp3 that has the recording. Please provide just the page numbers as a comma-delimited list. And please provide the list in ascending order.
""",
    }
);

### YouTube Video

In [ ]:
# gaia.invoke({
#     "task_id": "a1e91b78-d3d8-4675-bb8d-62741b4b68a6",
#     "question": "In the video https://www.youtube.com/watch?v=L1vXCYZAYYM, what is the highest number of bird species to be on camera simultaneously?"
# });

gaia.invoke(
    {
        "task_id": "9d191bce-651d-4746-be2d-7ef8ecadb9c2",
        "question": "Examine the video at https://www.youtube.com/watch?v=1htKBjuUWec. What does Teal'c say in response to the question \"Isn't that hot?\"",
    }
);

### Data Analysis

In [ ]:
gaia.invoke(
    {
        "task_id": "7bd855d8-463d-4ed5-93ca-5fe35145f733",
        "question": "The attached Excel file contains the sales of menu items for a local fast-food chain. What were the total sales that the chain made from food (not including drinks)? Express your answer in USD with two decimal places.",
    }
);

### Web Search

In [ ]:
# gaia.invoke({
#     "task_id": "8e867cd7-cff9-4e6c-867a-ff5ddc2550be",
#     "question": "How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia."
# });
gaia.invoke(
    {
        "task_id": "3f57289b-8c60-48be-bd80-01f8099ca449",
        "question": "How many at bats did the Yankee with the most walks in the 1977 regular season have that same season?",
    }
);

### Chess Problem

In [ ]:
gaia.invoke(
    {
        "task_id": "cca530fc-4052-43b2-b130-b30968d8aa44",
        "question": "Review the chess position provided in the image. It is black's turn. Provide the correct next move for black which guarantees a win. Please provide your response in algebraic notation.",
    }
);

## Errors Correction

In [ ]:
# gaia.invoke({
#     "task_id": "6f37996b-2ac7-44b0-8e68-6d28256631b4", # refinement overrides correct answer
#     "question": """\
# Given this table defining * on the set S = {a, b, c, d, e}

#   |*|a|b|c|d|e|
#   |---|---|---|---|---|---|
#   |a|a|b|c|b|d|
#   |b|b|c|a|e|c|
#   |c|c|a|b|b|a|
#   |d|b|e|b|e|d|
#   |e|d|b|a|d|c|

#   provide the subset of S involved in any possible counter-examples that prove * is not commutative. Provide your answer as a comma separated list of the elements in the set in alphabetical order.
# """
# });

# gaia.invoke({ # Refinement tend to override to Bartek
#     "task_id": "305ac316-eef6-4446-960a-92d80d542f82",
#     "question": "Who did the actor who played Ray in the Polish-language version of Everybody Loves Raymond play in Magda M.? Give only the first name."
# });

# gaia.invoke({ # Capitalized first list item
#     "task_id": "99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3",
#     "question": """\
# Hi, I'm making a pie but I could use some help with my shopping list. I have everything I need for the crust, but I'm not sure about the filling. I got the recipe from my friend Aditi, but she left it as a voice memo and the speaker on my phone is buzzing so I can't quite make out what she's saying. Could you please listen to the recipe and list all of the ingredients that my friend described? I only want the ingredients for the filling, as I have everything I need to make my favorite pie crust. I've attached the recipe as Strawberry pie.mp3.
# In your response, please only list the ingredients, not any measurements. So if the recipe calls for "a pinch of salt" or "two cups of ripe strawberries" the ingredients on the list would be "salt" and "ripe strawberries".
# Please format your response as a comma separated list of ingredients. Also, please alphabetize the ingredients.
# """
# });

# gaia.invoke(
#     {  # Lost list item during refinement and capitalized first list item
#         "task_id": "3cef3a44-215e-4aed-8e3b-b1e3f08063b7",
#         "question": """\
# I'm making a grocery list for my mom, but she's a professor of botany and she's a real stickler when it comes to categorizing things. I need to add different foods to different categories on the grocery list, but if I make a mistake, she won't buy anything inserted in the wrong category. Here's the list I have so far:
# milk, eggs, flour, whole bean coffee, Oreos, sweet potatoes, fresh basil, plums, green beans, rice, corn, bell pepper, whole allspice, acorns, broccoli, celery, zucchini, lettuce, peanuts
# I need to make headings for the fruits and vegetables. Could you please create a list of just the vegetables from my list? If you could do that, then I can figure out how to categorize the rest of the list into the appropriate categories. But remember that my mom is a real stickler, so make sure that no botanical fruits end up on the vegetable list, or she won't get them when she's at the store. Please alphabetize the list of vegetables, and place each item in a comma separated list.
# """,
#     }
# );

# gaia.invoke(
#     {  # Wrong agent selected
#         "task_id": "5a0c1adf-205e-4841-a666-7c3ef95def9d",
#         "question": "What is the first name of the only Malko Competition recipient from the 20th Century (after 1977) whose nationality on record is a country that no longer exists?",
#     }
# );

# gaia.invoke(
#     {  # Is it invalid ground truth in benchmark?
#         "task_id": "a0c07678-e491-4bbc-8f0b-07405144218f",
#         "question": """\
# Who are the pitchers with the number before and after Taish\u014D Tamai's number as of July 2023?
# Give them to me in the form Pitcher Before, Pitcher
# After, use their last names only, in Roman characters.
# """,
#     }
# );
